## import libraries

In [2]:
import sympy

## main set up of constants and functions

In [1]:

#constants
dim1 = 10 #dim 1 of box
dim2 = dim1 #dim 2 of box (make it square)
lj = 2 #length of jack
mj = 1 #mass of the bulb at each end of jack
mb = 25 #give the walls of the box some large mass relative to jack
g = 9.8

